In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
import requests
from bs4 import BeautifulSoup

import unicodedata
from lxml.html import parse
from urllib.request import urlopen

import os
import re
import json
from util import get

In [2]:
def get_introduction(soup):
    
    if soup.select('p')[0].get_text() == '抱歉，您所访问的页面不存在...':
        return 0
    
    text = ''
    for i in soup.select("div.lemma-summary"):
        for c in i:
            if c != '\n':
                for t in [t for t in c.children if t.name != 'sup']:  #去掉【1】这种类型的标签
                    text += t.string
                
    return text


def get_basic_info(soup):
    basic_info = {}
    try:
        for i in range(len(soup.select('div.basic-info')[0].select('dt'))) :
            basic_info[soup.select('div.basic-info')[0].select('dt')[i].get_text()
                      ] = soup.select('div.basic-info')[0].select('dd')[i].get_text()[1:-1]
    except IndexError:
        return np.nan
            
    return basic_info


def get_name(soup):
    return soup.select('div > dl > dd > h1')[0].string


def parse( url):
    if get(url) is None:
        return
    soup = BeautifulSoup(get(url).content.decode(), 'lxml')
    dic = {}
    dic['url'] = url
    dic['name'] = get_name(soup)
    dic['description'] = get_introduction(soup)
    dic['metadata'] = get_basic_info( soup)
    return dic

In [3]:
url_list = []
with open("./companies_baidubaike/companies_baidubaike.txt") as url_file:
    for line in url_file.readlines():
        url_list.append(line)

os.mkdir("./companies_baidubaike/json_folder")
os.chdir("./companies_baidubaike/json_folder")

for u in url_list:
    try:
        with open(u.split('/')[-1]+'.json', 'w') as json_file:
            json.dump(parse(u), json_file, indent=4)
    except Exception:
        pass